<a href="https://colab.research.google.com/gist/tims-prog/9470445ebe21d868470f7cbc3f98d96d/m1mini-python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using Python with the JADAK SkyeTek M1-Mini HF RFID Reader

This Application Note Example shows how to use the Python programming language to connect to a JADAK Camera (running J-Pro firmware). This example presumes that you have Python v3.8.X installed with a local jupyter notebook server running and a "VIBE" camera attached to the local machine.

    **NOTE:** *Examples require Python v3.8.X and the packages listed below*

#Environment Setup
 
**Python 3.8.X for Windows**

https://www.python.org/ftp/python/3.8.5/python-3.8.5-amd64.exe


Select a 'custom install' 
Choose the 'py' launcher and install all custom options. Install for all users on the machine, and to keep things simple, place the install in the C:\Program Files directory.  

**NOTE:** *During the installation process make sure you check the box for adding Python to your environment variable PATH.*

Once the above has been installed for all users on the machine, verify your system PATH env variable is set correctly.

Select 'This PC' in Windows 10 File Explorer, right click 'Properties' ->  Advanced system settings, then selecting Environment Variables.

Modify the System variables to ensure 

The install locations can vary:  
   e.g. if you installed Python as 'Administrator'
  
    C:\Program Files (x86)\Python38   
    C:\Program Files (x86)\Python38\Scripts
    C:\Program Files (x86)\Python36-32   
    C:\Program Files (x86)\Python36-32\Scripts

For this exercise, we need Python 3.8 to be first in the search PATH


Start up a Windows cmd/Powershell 'Run as administrator', then use the Python3 installer ***pip3*** to install jupyter notebook and the other required Python packages.  Use the provided 'requirements.txt' file for the package list

NOTE: You can get the latest binary installer of WIndows PowerShell here:
     https://aka.ms/powershell

    PS C:\Users\jadakAE>  pip3 install -r requirements.txt 

    PS C:\WINDOWS\system32> jupyter serverextension enable --py jupyter_http_over_ws

Edit the startJupyter.bat file to point to the local file location of the Example Directory. Then running as 'Administrator' use the Windows cmd and or powershell to start the local jupyter server.  

    PS C:\WINDOWS\system32> startJupyter.bat

Copy the line that appears once the server starts, e.g:
*http://localhost:8888/?token=3d969668917bea8baa95294337d6a87652f82fe25f596d05*
  

Since we are access local machine resources (USB Connected RFID Readers and or Cameras) we need to redirect the Google Colab Notebook to use a local Python installation.   Jupyter Notebook takes your local Python installation and wraps it with a web server.   You can connect "locally" to this server and access the attached Reader or Camera, or you can connect in "hosted" mode and run code using the Google Colab servers.

In the upper r.h. corner of the Google Collab window there will be an option to connect to a locally hosted server.   Select that option and paste the unique URL for the server running on your PC (similar to the URL above)

More info can be found here:URL: https://research.google.com/colaboratory/local-runtimes.html

#Package Imports and Background Functions
To begin, we make all necessary python package imports (Lines 1-6).

After importing the packages, we will then define a number of helper functions that will enable us to use Python with our Reader.

In [ ]:
import serial
import time
import random
import binascii
import sys
import glob


# CRC and Finish Frame
The first three helper functions we define are to help us add CRC codes to any command we want to pass to the Reader. CRC codes are required to send the commands using binary, so this is essential.

Our first helper function, stp2_crc16 (Lines 10 - 21) creates a CRC code. 

Our second helper function, finish_frame, (Lines 24 - 37) adds it to a requested byte array object (which will be our command to send to the Reader).

Our third helper function, match_crc16 (Lines 39 - 53) tests if CRC codes match what is expected from the Reader, and returns true or false, based on the result.

In [ ]:
#Handles Cyclical Redundancy Check
def stp2_crc16(data: bytes, poly=0x8408):
    data = bytearray(data)
    crc_16 = 0x0
    for b in data:
        crc_16 ^= b
        for _ in range(0, 8):
            if crc_16 & 0x0001:
                crc_16 >>= 1
                crc_16 ^= poly
            else:
                crc_16 >>= 1
    return crc_16 & 0xFFFF

#Finishes a Command Frame by adding the CRC to a command.
def finish_frame(p_frm: bytearray):

    # Add STP2 CRC16
    f_crc = stp2_crc16(p_frm)
    c_field = f_crc.to_bytes(2, byteorder='little')
    p_frm.append(c_field[1])
    p_frm.append(c_field[0])

    # Add <STX> to start of frame
    b_frm = bytearray()
    # <STX> or 0x02
    b_frm.append(0x02)
    b_frm.extend(p_frm)
    return b_frm
    
#Check for CRC Match
def match_crc16(resp: bytes):
    rsp_b = bytearray()
    # delete <STX>
    rsp_b = resp[1:]
    # delete crc16 or 2 bytes
    rsp_b = rsp_b[:len(rsp_b)-2]

    # Compute crc16 over response
    r_crc = stp2_crc16(rsp_b)

    c_field = r_crc.to_bytes(2, byteorder='little')
    if c_field[1] == resp[len(resp)-2] and c_field[0] == resp[len(resp)-1]:
        return True
    else:
        return False

#Memory Block Helpers
There are a couple of helper functions for reading and displaying data from a tag.

First is get_mem_block, (Lines 1-7) which takes a response from the Reader, and processes it to find and store the data. This is a function that is used to print the data we obtain from the reader. Since the FRAM tag is based on the Fujitsu MB89RC118C IC, the memory block size is 8 bytes. (http://www.fujitsu.com/downloads/MICRO/fme/fram/ds-mb89r118c.pdf)

Second is print_mem_block (Lines 10 - 17). This is a function which prints data, received from the function above.

Finally, we have reverse_mem_block (Lines 21 - 24) which reverses a set of data.

In [ ]:
#Takes data out of the tag reply.
def get_mem_block(rsp: bytearray):
    bid = bytearray()
    #Fujitsu pads echoed back cmd with 00, thus we start reading data at 4 (instead of 3)
    bid = rsp[4:]
    bid = bid[:len(bid)-2]
    return bid

#Print a received memory block from the tag.
def print_mem_block(bid: bytearray):
    print(' ')
    print("Memory Block: ", end=" ")
    for i in range(0,len(bid)):
        print(hex(bid[i]), end=" ")
        if i > 0 and i % 8 == 0:
            print(' ')
    print(' ')
    

#Reverse a memory block from the tag.    
def reverse_mem_block(bid: bytearray):
    swap_data = bid
    swap_data.reverse()
    return bid

# Extra Helper Functions
There are a number of miscellaneous helper functions documented here.

dump_frame (Lines 1-7) prints codes sent to and received from the reader.

is_command_success (Lines 10-14) checks if the command was a success by comparing the sent command and the response from the reader. If the codes both contain the command value at the right positions, it returns as true.

get_tag_identifier (Lines 17-21) gets the tag id from a Reader response, so we can save it and use it in other commands.

print_tag_id (Lines 23-29) prints the tag id obtained in the previous function.

In [ ]:
def dump_frame(data: bytearray, frm_type='Cmd: '):    
    print(frm_type + 'frame length:  ', len(data))
    print(frm_type, end=" ")
    for i in range(0, len(data)):
        print(hex(data[i]), end=" ")
        if i > 0 and i % 8 == 0:
            print(' ')


def is_cmd_success(cmd: bytearray, rsp: bytearray):
    if cmd[3] == rsp[2]:
        return True
    else:
        return False


def get_tag_identifier(rsp: bytearray):
    tid = bytearray()
    tid = rsp[3:]
    tid = tid[:len(tid)-2]
    return tid

def print_tag_id(tid: bytearray):
    print("TID: ", end=" ")
    for i in range(0,8):
        print(hex(tid[i]), end=" ")
        if i > 0 and i % 8 == 0:
            print(' ')
    print(' ')

#Send Command Helper Function
The send_command helper function sends a built command to the reader.

The actual sending is done via the Serial function "write" on Line 7.

We obtain the response via the Serial function "read" on Line 9.

We print the response (done on Line 13), then we check the CRC and verify the command's success (done in lines 14-22).

In [ ]:
def send_command(ser: serial.Serial, cmd: bytearray, rsp_len: int):

    print("Sending Command to Reader...")
    final_resp = bytearray()
    dump_frame(cmd, "Cmd: ")

    wrt_count = ser.write(cmd)

    resp = ser.read(rsp_len)
    if len(resp) > 0:
        rsp_frm = bytearray()
        rsp_frm.extend(resp)
        dump_frame(rsp_frm, "Rsp: ")
        if match_crc16(resp):
            if is_cmd_success(cmd, rsp_frm):
                final_resp = rsp_frm
            else:
                print("Cmd Failed: ", hex(resp[2]))
        else:
            print("Rsp: CRC does not match")
    else:
        print("No response from reader!")
        
    return final_resp

# Extra Functions
There a multitude of functions not used in this walkthrough, but included here for documentation's sake. The code here should be run to ensure the lab works as intended, but none of these functions will be used.

In [ ]:
#Generate a Random Number command.
def do_get_randnum(tag_id: bytearray, RF_F=False):
    exp_rsp_len = 7
    frame = bytearray()

    #Msg Length
    frame.append(0x0d)
    #Flags
    frame.append(0x68)
    #Get_RandNum_Tag
    frame.append(0x51)
    #Tag Type
    frame.append(0x01)
    #Tag Identifier
    frame.extend(tag_id)

    b_frm = finish_frame(frame)
    return b_frm, exp_rsp_len


def do_write_password(tag_id: bytearray, passwd: bytearray, RF_F=False):
    exp_rsp_len = 5
    frame = bytearray()

    #Msg Length
    frame.append(0x12)
    #Flags
    frame.append(0x48)
    #Write_Password_Tag
    frame.append(0x53)
    #Tag Type
    frame.append(0x01)
    #Tag Identifier
    frame.extend(tag_id)
    #Password Identifier
    frame.append(0x02)
    #Write password
    frame.extend(passwd)

    b_frm = finish_frame(frame)   
    return b_frm, exp_rsp_len


def do_set_password(tag_id: bytearray, passwd: bytearray, pwd_id=0x02, RF_F=False):   
    exp_rsp_len = 5
    frame = bytearray()

    #Msg Length
    frame.append(0x12)
    #Flags
    frame.append(0x48)
    #Set_Password_Tag
    frame.append(0x52)
    #Tag Type
    frame.append(0x01)
    #Tag Identifier
    frame.extend(tag_id)
    #Password Identifier
    frame.append(pwd_id)
    #Write password
    frame.extend(passwd)

    b_frm = finish_frame(frame)
    return b_frm, exp_rsp_len


def do_protect_paqe(tag_id: bytearray, RF_F=False):
    exp_rsp_len = 5
    frame = bytearray()

    #Msg Length
    frame.append(0x0f)
    #Flags
    frame.append(0x68)
    #Protect_Page_Tag
    frame.append(0x54)
    #Tag Type
    frame.append(0x0C)
    #Tag Identifier
    frame.extend(tag_id)
    #Page Number
    frame.append(0x09)
    #Page Status
    frame.append(0x10)

    b_frm = finish_frame(frame)
    return b_frm, exp_rsp_len


def do_lock_passwd(tag_id: bytearray, RF_F=False):
    exp_rsp_len = 5
    frame = bytearray()

    #Msg Length
    frame.append(0x0e)
    #Flags
    frame.append(0x68)
    #Get_RandNum_Tag
    frame.append(0x55)
    #Tag Type
    frame.append(0x01)
    #Tag Identifier
    frame.extend(tag_id)
    #Password Identifier
    frame.append(0x08)

    b_frm = finish_frame(frame)
    return b_frm, exp_rsp_len

    #Generate a System Info command.
def do_get_systeminfo(tag_id: bytearray, RF_F=False):
    exp_rsp_len = 18
    frame = bytearray()

    #Msg Length
    frame.append(0x0d)
    #Flags
    frame.append(0x68)
    #Get_System_Info_Tag
    frame.append(0x50)
    #Tag Type
    frame.append(0x01)
    #Tag Identifier
    frame.extend(tag_id)
    b_frm = finish_frame(frame)
    return b_frm, exp_rsp_len

#Firmware Version Check

With all the helper functions established, the first code which will be sent to the reader is Read_Sys, aka the Get Firmware Version command. We build this command using the below code.

Each function that builds a command for the reader builds it by adding hex values to a chain, adding a CRC to the end, and then returning the entire byte array, as well as an "expected response length".

In [ ]:
def get_firmware_version():
    exp_rsp_len = 7
    frame = bytearray()

    # Msg Length
    frame.append(0x06)
    # Flags
    frame.append(0x20)
    # Read_Sys (0x22)
    frame.append(0x22)
    # Starting Block - Firmware version (0x01)
    frame.append(0x01)
    # Number of Blocks
    frame.append(0x01)

    b_frm = finish_frame(frame)
    return b_frm, exp_rsp_len

Now that we have a get firmware version command, we can test it out in a main function.

Lines 1-8 are imports for everything needed in this main function.

Lines 12-26 identify and tell Python which COMport to use, as well as assign that COMport to a Serial object with a Baud Rate of 9600.

Lines 29-38 test out our get_firmware_version function. Once created, we send the command to the reader on Line 33, with send_command, which also processes our response.

In [ ]:
import serial
import glob
import io
import time
import random
import binascii
import sys
import serial.tools.list_ports


print(" ")
print("Serial Ports:")

#Change this serial string identifier to match hardware.
#
#COMport = str(next(serial.tools.list_ports.grep("USB Serial")))
#COMport = str(next(serial.tools.list_ports.grep("CH340")))
COMport = str(next(serial.tools.list_ports.grep("Prolific")))
    
print(COMport)
print(" ")
rfidCOMport = COMport.split()
#Create a Serial port object for computer to tag communication.
ser = serial.Serial(rfidCOMport[0], 9600, timeout=5)


# Get the Tag's firmware version. If this fails, System Exits with "Failed to get firmware version".
print("<<  Get FW version  >>")
frm, rsp_len = get_firmware_version()

response = send_command(ser, frm, rsp_len)
if len(response) == 0:
  ser.close()
  sys.exit("Exiting: Failed to get firmware version.")

time.sleep(1)

ser.close()

 
Serial Ports:
COM13 - Prolific USB-to-Serial Comm Port (COM13)
 
<<  Get FW version  >>
Sending Command to Reader...
Cmd: frame length:   8
Cmd:  0x2 0x6 0x20 0x22 0x1 0x1 0xa 0x19 Rsp: frame length:   7
Rsp:  0x2 0x5 0x22 0x9 0xd 0xd4 0x29 

#Select Tag

The next code to send is Select Tag. We build this command using the below code.

Like the previous function, we build the command by adding hex values to a chain, adding a CRC to the end, and then returning it and the expected response length.

Unlike the previous command, the last hex code in this command is a tag type, in which we use 0C, which refers to the FRAM tag type.



In [ ]:
#Generate the Select Tag command.
def do_select(RF_F=False):
    exp_rsp_len = 13
    frame = bytearray()

    # Msg Length
    frame.append(0x05)
    if RF_F:
        # Flags
        frame.append(0x20)
    else:
        # Flags
        frame.append(0x20)

    # Select_Tag
    frame.append(0x14)
    # Tag Type
    frame.append(0x0C)

    b_frm = finish_frame(frame)

    return b_frm, exp_rsp_len

Returning to the main function, we use our do_select function with our send_command function to get the reader to read and return a tag id. (Starting at Line 16)

In [ ]:
print(" ")
print("Serial Ports:")

#Change this serial string identifier to match hardware.
#
#COMport = str(next(serial.tools.list_ports.grep("USB Serial")))
#COMport = str(next(serial.tools.list_ports.grep("CH340")))
COMport = str(next(serial.tools.list_ports.grep("Prolific")))
    
print(COMport)
print(" ")
rfidCOMport = COMport.split()
#Create a Serial port object for computer to tag communication.
ser = serial.Serial(rfidCOMport[0], 9600, timeout=5)

# Turn on RF, and select the tag. If this fails, System Exits with "Failed to Select Tag".
RF_F = True
print(" ")
print(" ")
print("<<  Turning on RF and Selecting Tag >>")
frm, rsp_len = do_select(RF_F)
response = send_command(ser, frm, rsp_len)
if len(response) == 0:
  ser.close()
  sys.exit("Exiting: Failed to Select Tag")
tag_id = get_tag_identifier(response)
print(" ")
print_tag_id(tag_id)
time.sleep(1)

ser.close()

 
Serial Ports:
COM13 - Prolific USB-to-Serial Comm Port (COM13)
 
 
 
<<  Turning on RF and Selecting Tag >>
Sending Command to Reader...
Cmd: frame length:   7
Cmd:  0x2 0x5 0x20 0x14 0xc 0x55 0xf1 Rsp: frame length:   13
Rsp:  0x2 0xb 0x14 0xe0 0x8 0x1 0x47 0xfd 0xe2  
0x53 0xcd 0x4a 0x20  
TID:  0xe0 0x8 0x1 0x47 0xfd 0xe2 0x53 0xcd  


#Read Tag Block
Now that we have a tag ID, we can read that tag's data. Our next function creates the command that tells the reader to get an amount of info from the tag.

Like the previous function, we build the command by adding hex values to a chain, adding a CRC to the end, and then returning it and the expected response length.

Unlike the previous functions, this function has user defined parameters, which allow the function to read more than 1 block, and start from any block on the tag.

In [ ]:
#do_read_tag_block
#Generate a Read Tag Block command.
#Parameters:
#tag_id: The bytearray which represents the tag.
#RF_F: Is RF true or false? Defaults to False.
#start_block: Decimal integer that represents starting block.
#length_block: Decimal integer that represents blocks length to read.
#bytes_per_block: integer that represents bytes per block for a specific tag (4 or 8)
def do_read_tag_block(start_block, length_block, bytes_per_block, tag_id: bytearray, RF_F=False):
    
    exp_rsp_len = 6+(bytes_per_block*length_block)

    frame = bytearray()
    #Msg Length
    frame.append(0x0F)
    #Flags
    frame.append(0x60)
    #Read_Tag
    frame.append(0x24)
    #Tag Type
    frame.append(0x0C)
    #Tag Identifier
    frame.extend(tag_id)
    #Start Block
    frame.append(int(hex(start_block),0))
    #Number of Blocks to Read
    frame.append(int(hex(length_block),0))
    b_frm = finish_frame(frame)
    return b_frm, exp_rsp_len

Returning to the main function, we use our do_read_tag_block function with our send_command function to get the reader to read the tag.

For our user input parameters, we input 0 for starting block (first available index), we input 1 for number of blocks to read, and 8 for the number of bytes per block (as this is an FRAM tag that stores data in blocks of 8 bytes.) The tag id is also passed through to do_read_tag_block.

Reading the tag starts at line 16.

In [ ]:
print(" ")
print("Serial Ports:")

#Change this serial string identifier to match hardware.
#
#COMport = str(next(serial.tools.list_ports.grep("USB Serial")))
#COMport = str(next(serial.tools.list_ports.grep("CH340")))
COMport = str(next(serial.tools.list_ports.grep("Prolific")))
    
print(COMport)
print(" ")
rfidCOMport = COMport.split()
#Create a Serial port object for computer to tag communication.
ser = serial.Serial(rfidCOMport[0], 9600, timeout=5)

#Read the first block of the tag. If this fails, System Exits with "Failed to Read".
print(" ")
print(" ")
print("<<  Reading Blocks >>")
frm, rsp_len = do_read_tag_block(0, 1, 8, tag_id, RF_F)
response = send_command(ser, frm, rsp_len)
if len(response) == 0:
  ser.close()
  sys.exit("Exiting: Failed to Read")
data = get_mem_block(response)
print_mem_block(data)
time.sleep(1)

ser.close()

 
Serial Ports:
COM13 - Prolific USB-to-Serial Comm Port (COM13)
 
 
 
<<  Reading Blocks >>
Sending Command to Reader...
Cmd: frame length:   17
Cmd:  0x2 0xf 0x60 0x24 0xc 0xe0 0x8 0x1 0x47  
0xfd 0xe2 0x53 0xcd 0x0 0x1 0xf8 0xda  
Rsp: frame length:   14
Rsp:  0x2 0xc 0x24 0x0 0x0 0x0 0x0 0x0 0x0  
0x0 0x0 0x0 0xfe 0xd6  
Memory Block:  0x0 0x0 0x0 0x0 0x0 0x0 0x0 0x0  


#Write to Tag
After reading data, the next function creates the command that tells the reader to get change blocks of data on the tag.

Like the previous function, we build the command by adding hex values to a chain, adding a CRC to the end, and then returning it and the expected response length.

This function has user defined parameters, particularly which block to overwrite and what data to write to said block.

In [ ]:
#Generate a Write Tag Block command. This command can only write one block at a time.
#Parameters:
#tag_id: The bytearray which represents the tag.
#RF_F: Is RF true or false? Defaults to False.
#writedata: A byte array which is the block of data to be written.
#start_block: Decimal integer that represents block to write.
def do_write_tag_block(start_block, tag_id: bytearray, writedata: bytearray, RF_F=False):
    exp_rsp_len = 5
    frame = bytearray()
    #Msg Length 23 bytes (in hex)
    frame.append(0x17)
    #Flags
    frame.append(0x60)
    #Read_Tag
    frame.append(0x44)
    #Tag Type
    frame.append(0x0C)
    #Tag Identifier
    frame.extend(tag_id)
    #Start Block
    frame.append(int(hex(start_block),0))
    #Length to Read - This is fixed at 1.
    frame.append(0x01)
    #Data
    frame.extend(writedata)
    b_frm = finish_frame(frame)
    return b_frm, exp_rsp_len

Returning to the main function, we use our do_write_tag_block function with our send_command function to get the reader to read the tag.

For our user input parameters, we input 0 for starting block (first available index). 

The tag id is passed through to do_read_tag_block.

The data we write to the tag (writedata) is a reversed string of the data read by the previous functions, in which the tag read the first block of data. This functionality can be changed by commenting out line 6, and uncommenting line 8.

Writing to the tag starts at line 16.

In [ ]:
print(" ")
print("Serial Ports:")

#Change this serial string identifier to match hardware.
#
#COMport = str(next(serial.tools.list_ports.grep("USB Serial")))
#COMport = str(next(serial.tools.list_ports.grep("CH340")))
COMport = str(next(serial.tools.list_ports.grep("Prolific")))
    
print(COMport)
print(" ")
rfidCOMport = COMport.split()
#Create a Serial port object for computer to tag communication.
ser = serial.Serial(rfidCOMport[0], 9600, timeout=5)

#Write the first block of the tag. If this fails, System Exits with "Failed to Write".
print(" ")
print(" ") 
print("<<  Writing First Block >>")
#Reverse block 1 data to test. Then write it back into block 1.
writedata = reverse_mem_block(data)
#Write block 1 data
#writedata = bytearray([0xDE,0xAD, 0xBE, 0xEF, 0xAB, 0xCD, 0xEF, 0x00]) #DEADBEEFABCDEF00
    
frm, rsp_len = do_write_tag_block(0, tag_id, writedata, RF_F)
response = send_command(ser, frm, rsp_len)
if len(response) == 0:
  ser.close()
  sys.exit("Exiting: Failed to Write")
print(" ")
print("<< Block 1 Written >>")
time.sleep(1)    

ser.close()

 
Serial Ports:
COM13 - Prolific USB-to-Serial Comm Port (COM13)
 
 
 
<<  Writing First Block >>
Sending Command to Reader...
Cmd: frame length:   25
Cmd:  0x2 0x17 0x60 0x44 0xc 0xe0 0x8 0x1 0x47  
0xfd 0xe2 0x53 0xcd 0x0 0x1 0x0 0x0  
0x0 0x0 0x0 0x0 0x0 0x0 0x68 0x7f  
Rsp: frame length:   5
Rsp:  0x2 0x3 0x44 0x2e 0x48  
<< Block 1 Written >>


#SLIX 2 Differences
There are multiple differences when using the reader on newer firmware to identify SLIX-2 type tags.

Run the following code to verify the firmware version of the M1-Mini is 09E6. (and not 090D).



In [ ]:
import serial
import glob
import io
import time
import random
import binascii
import sys
import serial.tools.list_ports

#Main
if __name__ == "__main__":
    print(" ")
    print("Serial Ports:")
    
    #Change this serial string identifier to match hardware.
    #
    #COMport = str(next(serial.tools.list_ports.grep("USB Serial")))
    COMport = str(next(serial.tools.list_ports.grep("CH340")))
    #COMport = str(next(serial.tools.list_ports.grep("Prolific")))
    
    print(COMport)
    print(" ")
    rfidCOMport = COMport.split()
    #Create a Serial port object for computer to tag communication.
    ser = serial.Serial(rfidCOMport[0], 9600, timeout=5)


# Get the Tag's firmware version. If this fails, System Exits with "Failed to get firmware version".
    print("<<  Get FW version  >>")
    frm, rsp_len = get_firmware_version()

    response = send_command(ser, frm, rsp_len)
    if len(response) == 0:
        ser.close()
        sys.exit("Exiting: Failed to get firmware version.")

    time.sleep(1)

    ser.close()

 
Serial Ports:
COM12 - USB-SERIAL CH340 (COM12)
 
<<  Get FW version  >>
Sending Command to Reader...
Cmd: frame length:   8
Cmd:  0x2 0x6 0x20 0x22 0x1 0x1 0xa 0x19 Rsp: frame length:   7
Rsp:  0x2 0x5 0x22 0x9 0xe6 0x8d 0xf4 

After verifying the proper firmware, this block modifies all functions that need changing to work with SLIX-2.

The primary changes are using 01 instead of 0C in all "tag type" parameters, and changing the expected response lengths and byte array sizes of read and write functions to expect 4 bytes instead of 8 bytes per block. Lines that have specifically changed are outlined by "CHANGE -" comments.

In [ ]:
#Function modifications for SLIX-2 tags

#Generate the Select Tag command.
def do_select(RF_F=False):
    exp_rsp_len = 13
    frame = bytearray()

    # Msg Length
    frame.append(0x05)
    if RF_F:
        # Flags
        frame.append(0x28)
    else:
        # Flags
        frame.append(0x20)

    # Select_Tag
    frame.append(0x14)
    
    # CHANGE - Tag type is 01 instead of 0C.
    
    # Tag Type
    frame.append(0x01)

    b_frm = finish_frame(frame)

    return b_frm, exp_rsp_len

#Generate a System Info command.
def do_get_systeminfo(tag_id: bytearray, RF_F=False):
    exp_rsp_len = 18
    frame = bytearray()

    #Msg Length
    frame.append(0x0d)
    #Flags
    frame.append(0x68)
    #Get_System_Info_Tag
    frame.append(0x50)
     
    # CHANGE - Tag type is 01 instead of 0C.
    
    #Tag Type
    frame.append(0x01)
    #Tag Identifier
    frame.extend(tag_id)
    b_frm = finish_frame(frame)
    return b_frm, exp_rsp_len

#
#Generate a Write Tag Block command. This command can only write one block at a time.
#Parameters:
#tag_id: The bytearray which represents the tag.
#RF_F: Is RF true or false? Defaults to False.
#writedata: A byte array which is the block of data to be written.
#start_block: Decimal integer that represents block to write.
def do_write_tag_block(start_block, tag_id: bytearray, writedata: bytearray, RF_F=False):
    
    exp_rsp_len = 5
    frame = bytearray()
     
    # CHANGE - Message length is 19 instead of 23.
    
    #Msg Length 19 bytes (in hex)
    frame.append(0x13)
    #Flags
    frame.append(0x60)
    #Read_Tag
    frame.append(0x44)
     
    # CHANGE - Tag type is 01 instead of 0C.
    
    #Tag Type
    frame.append(0x01)
    #Tag Identifier
    frame.extend(tag_id)
    #Start Block
    frame.append(int(hex(start_block),0))
    #Length to Read - This is fixed at 1.
    frame.append(0x01)
    #Data
    frame.extend(writedata)
    b_frm = finish_frame(frame)
    return b_frm, exp_rsp_len

#do_read_tag_block
#Generate a Read Tag Block command.
#Parameters:
#tag_id: The bytearray which represents the tag.
#RF_F: Is RF true or false? Defaults to False.
#start_block: Decimal integer that represents starting block.
#length_block: Decimal integer that represents blocks length to read.
#bytes_per_block: integer that represents bytes per block for a specific tag (4 or 8)
def do_read_tag_block(start_block, length_block, bytes_per_block, tag_id: bytearray, RF_F=False):
     
    # CHANGE - Expected Response length is 1 less (5, not 6).
    
    exp_rsp_len = 5+(bytes_per_block*length_block)
    frame = bytearray()
    #Msg Length
    frame.append(0x0F)
    #Flags
    frame.append(0x60)
    #Read_Tag
    frame.append(0x24)
    #Tag Type
     
    # CHANGE - Tag type is 01 instead of 0C.
    
    frame.append(0x01)
    #Tag Identifier
    frame.extend(tag_id)
    #Start Block
    frame.append(int(hex(start_block),0))
    #Number of Blocks to Read
    frame.append(int(hex(length_block),0))
    b_frm = finish_frame(frame)
    return b_frm, exp_rsp_len

#Takes data out of the tag reply.
def get_mem_block(rsp: bytearray):
    bid = bytearray()
    bid = rsp[3:]
    bid = bid[:len(bid)-2]
    return bid

With these modifications, we can run the entire main process (Get Firmware, Select Tag, Read Tag, Write Tag) on a SLIX-2 tag, with the newer firmware version.

In [ ]:
import serial
import glob
import io
import time
import random
import binascii
import sys
import serial.tools.list_ports

#Main
if __name__ == "__main__":
    print(" ")
    print("Serial Ports:")
    
    #Change this serial string identifier to match hardware.
    #
    #COMport = str(next(serial.tools.list_ports.grep("USB Serial")))
    COMport = str(next(serial.tools.list_ports.grep("CH340")))
    #COMport = str(next(serial.tools.list_ports.grep("Prolific")))
    
    print(COMport)
    print(" ")
    rfidCOMport = COMport.split()
    #Create a Serial port object for computer to tag communication.
    ser = serial.Serial(rfidCOMport[0], 9600, timeout=5)


# Get the Tag's firmware version. If this fails, System Exits with "Failed to get firmware version".
    print("<<  Get FW version  >>")
    frm, rsp_len = get_firmware_version()

    response = send_command(ser, frm, rsp_len)
    if len(response) == 0:
        ser.close()
        sys.exit("Exiting: Failed to get firmware version.")

    time.sleep(1)


# Turn on RF, and select the tag. If this fails, System Exits with "Failed to Select Tag".
    RF_F = True
    print(" ")
    print(" ")
    print("<<  Turning on RF and Selecting Tag >>")
    frm, rsp_len = do_select(RF_F)
    response = send_command(ser, frm, rsp_len)
    if len(response) == 0:
        ser.close()
        sys.exit("Exiting: Failed to Select Tag")
    tag_id = get_tag_identifier(response)
    print(" ")
    print_tag_id(tag_id)
    time.sleep(1)

#Read the first block of the tag. If this fails, System Exits with "Failed to Read".
    print(" ")
    print(" ")
    print("<<  Reading Blocks >>")
    frm, rsp_len = do_read_tag_block(0, 1, 4, tag_id, RF_F)
    response = send_command(ser, frm, rsp_len)
    if len(response) == 0:
        ser.close()
        sys.exit("Exiting: Failed to Read")
    data = get_mem_block(response)
    print_mem_block(data)
    time.sleep(1)    

#Write the first block of the tag. If this fails, System Exits with "Failed to Write".
    print(" ")
    print(" ") 
    print("<<  Writing First Block >>")
    #Reverse block 1 data to test. Then write it back into block 1.
    writedata = reverse_mem_block(data)
    #Write block 1 data
    #writedata = bytearray([0xB0,0x00, 0xB1, 0xE5]) #B00B1E5
    
    frm, rsp_len = do_write_tag_block(0, tag_id, writedata, RF_F)
    response = send_command(ser, frm, rsp_len)
    if len(response) == 0:
        ser.close()
        sys.exit("Exiting: Failed to Write")
    print(" ")
    print("<< Block 1 Written >>")
    time.sleep(1)    

    if False:
        # Generate Get System Info command
        frm, rsp_len = do_get_systeminfo(tag_id, RF_F)

        response = send_command(ser, frm, rsp_len)

        if len(response) == 0:
            ser.close()
            sys.exit("Exiting 3")

        print("<<  Got System Info  >>")
        time.sleep(1)

    ser.close()

 
Serial Ports:
COM12 - USB-SERIAL CH340 (COM12)
 
<<  Get FW version  >>
Sending Command to Reader...
Cmd: frame length:   8
Cmd:  0x2 0x6 0x20 0x22 0x1 0x1 0xa 0x19 Rsp: frame length:   7
Rsp:  0x2 0x5 0x22 0x9 0xe6 0x8d 0xf4  
 
<<  Turning on RF and Selecting Tag >>
Sending Command to Reader...
Cmd: frame length:   7
Cmd:  0x2 0x5 0x28 0x14 0x1 0x48 0xd6 Rsp: frame length:   13
Rsp:  0x2 0xb 0x14 0xe0 0x4 0x1 0x8 0xf 0x52  
0xba 0x24 0xd0 0x35  
TID:  0xe0 0x4 0x1 0x8 0xf 0x52 0xba 0x24  
 
 
<<  Reading Blocks >>
Sending Command to Reader...
Cmd: frame length:   17
Cmd:  0x2 0xf 0x60 0x24 0x1 0xe0 0x4 0x1 0x8  
0xf 0x52 0xba 0x24 0x0 0x1 0x9f 0x4e  
Rsp: frame length:   9
Rsp:  0x2 0x7 0x24 0x0 0x0 0x0 0x0 0x51 0x50  
 
Memory Block:  0x0 0x0 0x0 0x0  
 
 
<<  Writing First Block >>
Sending Command to Reader...
Cmd: frame length:   21
Cmd:  0x2 0x13 0x60 0x44 0x1 0xe0 0x4 0x1 0x8  
0xf 0x52 0xba 0x24 0x0 0x1 0x0 0x0  
0x0 0x0 0x54 0x27 Rsp: frame length:   5
Rsp:  0x2 0x3 0x44 0x2